In [2]:
%load_ext lab_black

In [3]:
import pandas as pd
import numpy as np
from nltk import tokenize
from datetime import datetime
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from numba import jit, prange

In [7]:
nvidia_df = pd.read_csv("../00_data/Cleaned_Data_10MAR.gz", compression="gzip")

In [8]:
nvidia_df

,Unnamed: 0,Index,videoID,datePub,searchedDate,VideoTitle,channelTitle,viewCount,likeCount,dislikeCount,captionString
0,0,0,y4fb_R5Ogw0,2020-10-01 05:16:02+00:00,2020-10-02 01:45:21.553469+00:00,"2x NVIDIA RTX 3090 SLI Benchmarks: 500FPS, 700...",Gamers Nexus,367472,15701,260,so this setup that you're seeing is what we ha...
1,1,1,ALEXVtnNEwA,2020-09-01 17:15:57+00:00,2020-10-02 01:45:22.351801+00:00,NVIDIA GeForce Special Event Livestream,IGN,543115,16860,411,b welcome to my kitchen i hope all of you are ...
2,2,2,E98hC9e__Xs,2020-09-01 16:42:07+00:00,2020-10-02 01:45:23.252012+00:00,NVIDIA GeForce RTX 30 Series | Official Launch...,NVIDIA GeForce,1484075,54741,1135,Welcome to my kitchen. I hope all of you are...
3,3,3,nMns8t4OhI0,2020-09-30 23:00:06+00:00,2020-10-02 01:45:23.922084+00:00,NVIDIA Responds to RTX 3080 Crashes...,JayzTwoCents,561238,27766,885,NaN
4,4,4,AG_ZHi3tuyk,2020-09-16 12:59:59+00:00,2020-10-02 01:45:24.592244+00:00,"Nvidia, you PROMISED! - RTX 3080 Review",Linus Tech Tips,3391565,141852,3961,- The RTX 3080 is the most powerful GPU we've ...
...,...,...,...,...,...,...,...,...,...,...,...
16613,16613,20,cdSAJyIgmdI,2020-09-24 23:11:46+00:00,2021-02-20 21:30:39.712827+00:00,Jim Cramer breaks down Nvidia's acquisition of...,CNBC Television,66815,883,37,what do we do with the stock of nvidia now th...
16614,16614,21,YjcxrfEVhc8,2020-09-24 13:00:00+00:00,2021-02-20 21:30:40.360748+00:00,Making Nvidia’s CEO mad - RTX 3090 Review,Linus Tech Tips,3193067,142120,3797,- Nvidia's top of the line RTX 3090. (upbeat m...
16615,16615,22,iXn9O-Rzb_M,2020-12-12 03:10:26+00:00,2021-02-20 21:30:41.003955+00:00,NVIDIA might ACTUALLY be EVIL... - WAN Show De...,Linus Tech Tips,1697803,100343,1802,NaN
16616,16616,23,HiMy1C4XVrQ,2021-01-19 23:20:39+00:00,2021-02-20 21:30:41.521426+00:00,The GPU NVIDIA didn’t tell anyone about…,Paul's Hardware,102002,7340,126,hello everyone and welcome to paul's hardware...


In [5]:
nvidia_df["Date"].value_counts().sort_index()

KeyError: 'Date'

In [ ]:
nvidia_df.columns

In [9]:
# initialize correct types of data for each column
nvidia_df["VideoTitle"] = nvidia_df["VideoTitle"].astype("str")
nvidia_df["datePub"] = pd.to_datetime(
    nvidia_df["datePub"], format="%Y-%m-%d %H:%M:%S", utc=True
)
nvidia_df["searchedDate"] = pd.to_datetime(
    nvidia_df["searchedDate"], format="%Y-%m-%d %H:%M:%S", utc=True
)
nvidia_df["channelTitle"] = nvidia_df["channelTitle"].astype("str")
nvidia_df["viewCount"] = nvidia_df["viewCount"].astype("int")
nvidia_df["likeCount"] = nvidia_df["likeCount"].astype("int")
nvidia_df["dislikeCount"] = nvidia_df["dislikeCount"].astype("int")
nvidia_df["captionString"] = nvidia_df["captionString"].astype("str")

In [10]:
type(nvidia_df["VideoTitle"][0])

str

In [11]:
# X Variables = Aggregate the following values by search date/time
#     videoTitleSentimentScore_pos = vader pos label for video title
#     captionSentimentScore_pos = vader pos label for caption string
#     XXX - viewcounts_weighted = views count / number of day since published
#     XXX - like_count_weighted = like count / number of day since published
#     XXX - dislike_count_weighted = dislike count / number of day since published
#     XXX - like_dislike_ratio = like / (like + dislike)
#     XXX - weighted_like_dislike_ratio = weighted_like / (weighted_like + weighted_dislike)
#     XXX - like_dislike_ratio_by_viewcounts_weighted = like dislike ration (video potential) * viewcount_weight

In [12]:
# load vader sentiment for comparing the uncleaned strings to the modified ones
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [13]:
# print(f'Title to score = {nvidia_df["captionString"][5]}')
score = sid.polarity_scores(str(nvidia_df["captionString"][5]))
print(score)

{'neg': 0.037, 'neu': 0.846, 'pos': 0.117, 'compound': 0.9994}


In [14]:
nvidia_df["videoTitleSentimentScore_pos"] = [
    sid.polarity_scores(x)["pos"] for x in nvidia_df["VideoTitle"]
]

nvidia_df["videoTitleSentimentScore_neg"] = [
    sid.polarity_scores(x)["neg"] for x in nvidia_df["VideoTitle"]
]

nvidia_df["captionSentimentScore_pos"] = [
    sid.polarity_scores(x)["pos"] for x in nvidia_df["captionString"]
]

nvidia_df["captionSentimentScore_neg"] = [
    sid.polarity_scores(x)["neg"] for x in nvidia_df["captionString"]
]

In [15]:
nvidia_df.tail(10)

,Unnamed: 0,Index,videoID,datePub,searchedDate,VideoTitle,channelTitle,viewCount,likeCount,dislikeCount,captionString,videoTitleSentimentScore_pos,videoTitleSentimentScore_neg,captionSentimentScore_pos,captionSentimentScore_neg
16608,16608,15,eKH6no71-UA,2021-01-20 21:56:49+00:00,2021-02-20 21:30:36.718454+00:00,Nvidia RTX Stock Coming - GPU Shortages Possib...,Graphically Challenged,178378,6173,1188,nvidia has some plans that may finally put an ...,0.0,0.186,0.138,0.034
16609,16609,16,msmmdMrKPuY,2021-02-16 23:15:04+00:00,2021-02-20 21:30:37.316991+00:00,Nvidia & AMD’s Answer to Cryptocurrency Miners,Moore's Law Is Dead,49322,3906,103,to be entirely honest i found the whole conve...,0.0,0.000,0.127,0.055
16610,16610,17,I2icFFqzrIg,2021-02-20 06:00:02+00:00,2021-02-20 21:30:37.957025+00:00,"✅NVIDIA МОЛОДЦЫ, RTX 3060 без майнинга, беда с...",HOMISH,13918,1675,69,nan,0.0,0.000,0.000,0.000
16611,16611,18,Dw4oet5f0dI,2020-10-05 12:55:38+00:00,2021-02-20 21:30:38.496794+00:00,GPU Technology Conference (GTC) Keynote Oct 20...,NVIDIA,219512,1531,83,I am an explorer. Searching for the origins of...,0.0,0.000,0.167,0.020
16612,16612,19,fVbQeO8iJh4,2021-02-19 14:00:30+00:00,2021-02-20 21:30:39.128417+00:00,Nvidia Finally Does Something About Miners,Hot News,15918,1275,30,nvidia's decided that it's had it with miners ...,0.0,0.000,0.103,0.028
16613,16613,20,cdSAJyIgmdI,2020-09-24 23:11:46+00:00,2021-02-20 21:30:39.712827+00:00,Jim Cramer breaks down Nvidia's acquisition of...,CNBC Television,66815,883,37,what do we do with the stock of nvidia now th...,0.0,0.000,0.139,0.061
16614,16614,21,YjcxrfEVhc8,2020-09-24 13:00:00+00:00,2021-02-20 21:30:40.360748+00:00,Making Nvidia’s CEO mad - RTX 3090 Review,Linus Tech Tips,3193067,142120,3797,- Nvidia's top of the line RTX 3090. (upbeat m...,0.0,0.348,0.126,0.044
16615,16615,22,iXn9O-Rzb_M,2020-12-12 03:10:26+00:00,2021-02-20 21:30:41.003955+00:00,NVIDIA might ACTUALLY be EVIL... - WAN Show De...,Linus Tech Tips,1697803,100343,1802,nan,0.0,0.000,0.000,0.000
16616,16616,23,HiMy1C4XVrQ,2021-01-19 23:20:39+00:00,2021-02-20 21:30:41.521426+00:00,The GPU NVIDIA didn’t tell anyone about…,Paul's Hardware,102002,7340,126,hello everyone and welcome to paul's hardware...,0.0,0.000,0.122,0.041
16617,16617,24,MyEEMZNZvjA,2021-02-10 14:00:18+00:00,2021-02-20 21:30:42.127197+00:00,Nvidia Gave Up!,Hot News,60907,3427,108,hi friends welcome to your wednesday edition o...,0.0,0.000,0.108,0.084


In [16]:
nvidia_df.iloc[:, -2:]

,captionSentimentScore_pos,captionSentimentScore_neg
0,0.124,0.067
1,0.145,0.027
2,0.133,0.025
3,0.000,0.000
4,0.140,0.043
...,...,...
16613,0.139,0.061
16614,0.126,0.044
16615,0.000,0.000
16616,0.122,0.041


In [17]:
nvidia_df["num_days_since_pub"] = np.ceil(
    (nvidia_df["searchedDate"] - nvidia_df["datePub"]) / np.timedelta64(1, "D")
)
nvidia_df["viewcounts_weighted"] = (
    nvidia_df["viewCount"] / nvidia_df["num_days_since_pub"]
)
nvidia_df["like_count_weighted"] = (
    nvidia_df["likeCount"] / nvidia_df["num_days_since_pub"]
)
nvidia_df["dislike_count_weighted"] = (
    nvidia_df["dislikeCount"] / nvidia_df["num_days_since_pub"]
)
nvidia_df["like_dislike_ratio"] = nvidia_df["likeCount"] / (
    nvidia_df["likeCount"] + nvidia_df["dislikeCount"]
)
nvidia_df["weighted_like_dislike_ratio"] = nvidia_df["like_count_weighted"] / (
    nvidia_df["like_count_weighted"] + nvidia_df["dislike_count_weighted"]
)
nvidia_df["like_dislike_ratio_by_viewcounts_weighted"] = (
    nvidia_df["like_dislike_ratio"] * nvidia_df["viewcounts_weighted"]
)
nvidia_df["Date"] = pd.to_datetime(
    nvidia_df["searchedDate"], format="%Y-%m-%d", utc=True
).dt.date

In [18]:
nvidia_df.columns

Index(['Unnamed: 0', 'Index', 'videoID', 'datePub', 'searchedDate',
       'VideoTitle', 'channelTitle', 'viewCount', 'likeCount', 'dislikeCount',
       'captionString', 'videoTitleSentimentScore_pos',
       'videoTitleSentimentScore_neg', 'captionSentimentScore_pos',
       'captionSentimentScore_neg', 'num_days_since_pub',
       'viewcounts_weighted', 'like_count_weighted', 'dislike_count_weighted',
       'like_dislike_ratio', 'weighted_like_dislike_ratio',
       'like_dislike_ratio_by_viewcounts_weighted', 'Date'],
      dtype='object')

In [19]:
X_features = nvidia_df.filter(
    [
        "videoTitleSentimentScore_pos",
        "videoTitleSentimentScore_neg",
        "captionSentimentScore_pos",
        "captionSentimentScore_neg",
        "num_days_since_pub",
        "viewcounts_weighted",
        "like_count_weighted",
        "like_dislike_ratio",
        "weighted_like_dislike_ratio",
        "like_dislike_ratio_by_viewcounts_weighted",
        "Date",
    ],
    axis=1,
)

In [20]:
X_feautures_pre_merge = X_features.groupby(["Date"]).mean().reset_index()

In [32]:
X_feautures_pre_merge.tail()

,Date,videoTitleSentimentScore_pos,videoTitleSentimentScore_neg,captionSentimentScore_pos,captionSentimentScore_neg,num_days_since_pub,viewcounts_weighted,like_count_weighted,like_dislike_ratio,weighted_like_dislike_ratio,like_dislike_ratio_by_viewcounts_weighted
87,2021-02-15,0.034680,0.077251,0.107473,0.031818,115.607273,13347.728400,693.672590,0.956675,0.956675,12957.795705
88,2021-02-16,0.040633,0.051960,0.102418,0.032844,120.298182,25551.254491,1678.468242,0.960266,0.960266,25040.842574
89,2021-02-18,0.046834,0.058583,0.101794,0.032863,119.245714,16224.086329,939.047614,0.954441,0.954441,15837.504011
90,2021-02-19,0.033890,0.069657,0.097573,0.031320,113.623333,19150.754564,1316.566845,0.958693,0.958693,18586.708998
91,2021-02-20,0.045380,0.073320,0.103120,0.034700,101.600000,41163.221800,3604.290577,0.958253,0.958253,40312.584142


In [33]:
# This is a function to generate response variable dataframe
# Decision rule:
# If the change in "High" in 5 days is greater than 3%, we give it label "buy"
# If the change in "High" in 5 days is smaller than -3%, we give it label "sell"
# Otherwise, we give it label "hold
def create_response(start_date, end_date, stock_name):
    import pandas as pd
    import numpy as np
    import yfinance as yf

    # get stock info
    data = (
        yf.download(stock_name, start=start_date, end=end_date)
        .asfreq("D")
        .fillna(method="ffill")
    )
    # create percentage dict
    percent_dict = {}
    for i in range(len(data) - 5):
        cur_high = data.iloc[i, :]["High"]
        future_high = data.iloc[i + 5, :]["High"]
        percent_dict[data.index[i]] = (future_high - cur_high) / cur_high
    response_df = pd.DataFrame.from_dict(
        percent_dict, orient="index", columns=["percent_change"]
    )
    # Decsion rule
    bins = [-(np.inf), -0.03, 0.03, np.inf]
    names = ["sell", "hold", "buy"]
    response_df["action"] = pd.cut(response_df["percent_change"], bins, labels=names)
    response_df = response_df.reset_index()
    return response_df

In [34]:
responce_df = create_response("2020-10-02", "2021-02-28", "NVDA")

[*********************100%***********************]  1 of 1 completed


In [35]:
responce_df

,index,percent_change,action
0,2020-10-02,0.044362,buy
1,2020-10-03,0.041791,buy
2,2020-10-04,0.022541,hold
3,2020-10-05,0.013192,hold
4,2020-10-06,-0.022469,hold
...,...,...,...
138,2021-02-17,-0.005928,hold
139,2021-02-18,-0.042660,sell
140,2021-02-19,-0.044162,sell
141,2021-02-20,-0.052860,sell


In [36]:
X_feautures_pre_merge["Date"] = pd.to_datetime(
    X_feautures_pre_merge["Date"], format="%Y-%m-%d", utc=True
).dt.date

responce_df["index"] = pd.to_datetime(
    responce_df["index"], format="%Y-%m-%d", utc=True
).dt.date

In [37]:
type(X_feautures_pre_merge["Date"][0])
X_feautures_pre_merge["Date"].tail(50)

42    2020-11-21
43    2020-11-22
44    2020-11-23
45    2020-11-24
46    2020-11-25
47    2020-11-26
48    2020-11-30
49    2020-12-02
50    2020-12-05
51    2020-12-06
52    2020-12-07
53    2020-12-08
54    2020-12-12
55    2020-12-15
56    2020-12-18
57    2020-12-19
58    2020-12-20
59    2020-12-21
60    2020-12-22
61    2020-12-24
62    2020-12-27
63    2021-01-02
64    2021-01-04
65    2021-01-07
66    2021-01-08
67    2021-01-15
68    2021-01-16
69    2021-01-17
70    2021-01-18
71    2021-01-19
72    2021-01-20
73    2021-01-21
74    2021-01-23
75    2021-01-28
76    2021-01-29
77    2021-01-30
78    2021-02-01
79    2021-02-02
80    2021-02-04
81    2021-02-06
82    2021-02-07
83    2021-02-08
84    2021-02-10
85    2021-02-11
86    2021-02-14
87    2021-02-15
88    2021-02-16
89    2021-02-18
90    2021-02-19
91    2021-02-20
Name: Date, dtype: object

In [38]:
type(responce_df["index"][0])
responce_df["index"][0]

datetime.date(2020, 10, 2)

In [46]:
X_feautures_final = X_feautures_pre_merge.merge(
    responce_df, left_on="Date", right_on="index", how="left", indicator=True
)

In [47]:
X_feautures_final.to_csv("../00_data/Cleaned_Sent_Data_16MAR_v3.gz", compression="gzip")

### APPENDIX

In [ ]:
pd.DataFrame(
    columns=[
        "videoTitleSentimentScore_pos",
        "captionSentimentScore_pos",
        "viewcounts_weighted",
        "like_count_weighted",
        "dislike_count_weighted",
        "like_dislike_ratio",
        "like_dislike_ratio_by_viewcounts_weighted",
    ]
)

nvidia_df["videoTitleSentimentScore_pos"] = [
    sid.polarity_scores(x)["pos"] for x in nvidia_df["VideoTitle"]
]

nvidia_df["videoTitleSentimentScore_neg"] = [
    sid.polarity_scores(x)["neg"] for x in nvidia_df["VideoTitle"]
]

nvidia_df["captionSentimentScore_pos"] = [
    sid.polarity_scores(x)["pos"] for x in nvidia_df["captionString"]
]

nvidia_df["captionSentimentScore_neg"] = [
    sid.polarity_scores(x)["neg"] for x in nvidia_df["captionString"]
]

In [ ]:
X_features = pd.DataFrame(
    columns=[
        "videoTitleSentimentScore_pos",
        "captionSentimentScore_pos",
        "viewcounts_weighted",
        "like_count_weighted",
        "dislike_count_weighted",
        "like_dislike_ratio",
        "like_dislike_ratio_by_viewcounts_weighted",
    ]
)

In [ ]:
nvidia_df["num_days_since_pub"] = np.ceil(
    (nvidia_df["searchedDate"] - nvidia_df["datePub"]) / np.timedelta64(1, "D")
)

In [ ]:
nvidia_df["viewcounts_weighted"] = (
    nvidia_df["viewCount"] / nvidia_df["num_days_since_pub"]
)

In [ ]:
nvidia_df["like_count_weighted"] = (
    nvidia_df["likeCount"] / nvidia_df["num_days_since_pub"]
)

In [ ]:
nvidia_df["like_count_weighted"] = (
    nvidia_df["dislikeCount"] / nvidia_df["num_days_since_pub"]
)